In [2]:
from openbb import obb

obb.equity.price.historical("AAPL", start_date="2020-01-01", end_date="2020-01-31")

OBBject

id: 06823f27-1487-79a2-8000-bf5c9b981113
results: [{'date': datetime.date(2020, 1, 2), 'open': 74.05999755859375, 'high': 75...
provider: yfinance
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'}, 's...